In [1]:
pip install onnx numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install onnx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [10]:
pip install numba

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install numba

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from numba import cuda
print(cuda.detect())

Found 1 CUDA devices
id 0      b'NVIDIA T400 4GB'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-f748cbed-5ce7-bff1-bd01-ea1a21a5bf6f
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported
True


In [81]:
from PIL import Image

img_path = "pexels-magalie-parise-2147945619-32654307 (1).jpg"
img = Image.open(img_path).resize((224, 224)).convert("RGB")
img_np = np.array(img).astype(np.uint8)  # shape: (224, 224, 3)

In [82]:
img_nchw = img_np.transpose(2, 0, 1)  # shape: (3, 224, 224)

# Get input quantization parameters from ONNX
x_scale = get_array("data_scale")      # should be float32 scalar
x_zp = get_array("data_zero_point")    # should be uint8 scalar

# Quantize
input_q = np.clip(np.round(img_nchw / x_scale) + x_zp, 0, 255).astype(np.uint8)

In [83]:
import onnx
import numpy as np
from onnx import numpy_helper

# === Load the model ===
model_path = "vgg16_int8.onnx"
onnx_model = onnx.load(model_path)

# === Get initializers ===
initializer_map = {init.name: init for init in onnx_model.graph.initializer}
def get_array(name): return numpy_helper.to_array(initializer_map[name])

# === Extract QLinearConv nodes ===
qconv_nodes = [node for node in onnx_model.graph.node if node.op_type == "QLinearConv"]
layer_info = []

for i, node in enumerate(qconv_nodes[:2]):
    inputs = node.input[:8]  # First 8 inputs
    x_scale = get_array(inputs[1])
    x_zp = get_array(inputs[2])
    W = get_array(inputs[3])
    w_scale = get_array(inputs[4])
    w_zp = get_array(inputs[5])
    y_scale = get_array(inputs[6])
    y_zp = get_array(inputs[7])
    bias = get_array(node.input[8]) if len(node.input) > 8 else None
    layer_info.append({
        "W": W,
        "w_scale": w_scale,
        "w_zp": w_zp,
        "x_scale": x_scale,
        "x_zp": x_zp,
        "y_scale": y_scale,
        "y_zp": y_zp,
        "bias": bias
    })

# Print shape confirmation
print("Layer 1 Weights:", layer_info[0]['W'].shape)
print("Layer 2 Weights:", layer_info[1]['W'].shape)


Layer 1 Weights: (64, 3, 3, 3)
Layer 2 Weights: (64, 64, 3, 3)


In [49]:
from PIL import Image

img_path = "pexels-magalie-parise-2147945619-32654307 (1).jpg"
img = Image.open(img_path).resize((224, 224)).convert("RGB")
img_np = np.array(img).astype(np.uint8)  # shape: (224, 224, 3)


In [50]:
img_nchw = img_np.transpose(2, 0, 1)  # shape: (3, 224, 224)

# Get input quantization parameters from ONNX
x_scale = get_array("data_scale")      # should be float32 scalar
x_zp = get_array("data_zero_point")    # should be uint8 scalar

# Quantize
input_q = np.clip(np.round(img_nchw / x_scale) + x_zp, 0, 255).astype(np.uint8)


In [84]:
def manual_qconv2d(input_q, W_q, bias, x_scale, x_zp, w_scale, w_zp, y_scale, y_zp, padding=1):
    C_in, H, W = input_q.shape
    C_out, _, kH, kW = W_q.shape

    # Apply zero padding to the input
    input_padded = np.pad(input_q, ((0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=x_zp)

    # Output shape: HxW preserved
    output = np.zeros((C_out, H, W), dtype=np.int32)

    for oc in range(C_out):
        acc = np.zeros((H, W), dtype=np.int32)
        for ic in range(C_in):
            for i in range(kH):
                for j in range(kW):
                    acc += (
                        (input_padded[ic, i:i+H, j:j+W].astype(np.int32) - x_zp)
                        * (W_q[oc, ic, i, j].astype(np.int32) - w_zp[oc])
                    )
        if bias is not None:
            acc += int(np.round(bias[oc] / (x_scale * w_scale[oc])))

        out_q = np.round(acc * x_scale * w_scale[oc] / y_scale + y_zp).astype(np.uint8)
        output[oc] = np.clip(out_q, 0, 255)

    return output


In [85]:
# === Run the full Layer 1 pass with padding ===
layer1_out_q = manual_qconv2d(
    input_q=input_q,
    W_q=layer_info[0]['W'],
    bias=layer_info[0]['bias'],
    x_scale=layer_info[0]['x_scale'],
    x_zp=layer_info[0]['x_zp'],
    w_scale=layer_info[0]['w_scale'],
    w_zp=layer_info[0]['w_zp'],
    y_scale=layer_info[0]['y_scale'],
    y_zp=layer_info[0]['y_zp'],
    padding=1   # 👈 Enables 224×224 output
)

print("✅ Layer 1 Output Shape with padding:", layer1_out_q.shape)


✅ Layer 1 Output Shape with padding: (64, 224, 224)


In [86]:
def maxpool2d(input_tensor, kernel_size=2, stride=2):
    C, H, W = input_tensor.shape
    out_H = (H - kernel_size) // stride + 1
    out_W = (W - kernel_size) // stride + 1
    pooled = np.zeros((C, out_H, out_W), dtype=input_tensor.dtype)
    
    for c in range(C):
        for i in range(out_H):
            for j in range(out_W):
                h_start = i * stride
                h_end = h_start + kernel_size
                w_start = j * stride
                w_end = w_start + kernel_size
                pooled[c, i, j] = np.max(input_tensor[c, h_start:h_end, w_start:w_end])
    
    return pooled


In [87]:
# === Run Layer 2 ===
layer2_out_q = manual_qconv2d(
    input_q=layer1_out_q,
    W_q=layer_info[1]['W'],
    bias=layer_info[1]['bias'],
    x_scale=layer_info[1]['x_scale'],
    x_zp=layer_info[1]['x_zp'],
    w_scale=layer_info[1]['w_scale'],
    w_zp=layer_info[1]['w_zp'],
    y_scale=layer_info[1]['y_scale'],
    y_zp=layer_info[1]['y_zp'],
    padding=1  # 👈 Padding to maintain 224x224
)

print("✅ Layer 2 Output Shape:", layer2_out_q.shape)  # Expect (64, 224, 224)

# === Apply MaxPool ===
layer2_pooled_q = maxpool2d(layer2_out_q, kernel_size=2, stride=2)
print("✅ MaxPool Output Shape:", layer2_pooled_q.shape)  # Expect (64, 112, 112)


✅ Layer 2 Output Shape: (64, 224, 224)
✅ MaxPool Output Shape: (64, 112, 112)


In [88]:
import onnx
import numpy as np
from onnx import numpy_helper

# Load the model
model_path = "vgg16_int8.onnx"
onnx_model = onnx.load(model_path)

# Build initializer map
initializer_map = {init.name: init for init in onnx_model.graph.initializer}
def get_array(name): return numpy_helper.to_array(initializer_map[name])

# Extract all QLinearConv nodes
qconv_nodes = [node for node in onnx_model.graph.node if node.op_type == "QLinearConv"]

# Gather layer-wise data
layer_info = []

for i, node in enumerate(qconv_nodes):
    inputs = node.input[:8]
    layer_data = {
        "W": get_array(inputs[3]),
        "w_scale": get_array(inputs[4]),
        "w_zp": get_array(inputs[5]),
        "x_scale": get_array(inputs[1]),
        "x_zp": get_array(inputs[2]),
        "y_scale": get_array(inputs[6]),
        "y_zp": get_array(inputs[7]),
        "bias": get_array(node.input[8]) if len(node.input) > 8 else None
    }
    layer_info.append(layer_data)

# Optional: print weight shapes to confirm
for i, layer in enumerate(layer_info):
    print(f"Layer {i+1}: W shape = {layer['W'].shape}, bias shape = {None if layer['bias'] is None else layer['bias'].shape}")


Layer 1: W shape = (64, 3, 3, 3), bias shape = (64,)
Layer 2: W shape = (64, 64, 3, 3), bias shape = (64,)
Layer 3: W shape = (128, 64, 3, 3), bias shape = (128,)
Layer 4: W shape = (128, 128, 3, 3), bias shape = (128,)
Layer 5: W shape = (256, 128, 3, 3), bias shape = (256,)
Layer 6: W shape = (256, 256, 3, 3), bias shape = (256,)
Layer 7: W shape = (256, 256, 3, 3), bias shape = (256,)
Layer 8: W shape = (512, 256, 3, 3), bias shape = (512,)
Layer 9: W shape = (512, 512, 3, 3), bias shape = (512,)
Layer 10: W shape = (512, 512, 3, 3), bias shape = (512,)
Layer 11: W shape = (512, 512, 3, 3), bias shape = (512,)
Layer 12: W shape = (512, 512, 3, 3), bias shape = (512,)
Layer 13: W shape = (512, 512, 3, 3), bias shape = (512,)


In [89]:
# === Layer 3 Convolution ===
layer3_out_q = manual_qconv2d(
    input_q=layer2_pooled_q,                      # (64, 112, 112)
    W_q=layer_info[2]['W'],                       # (128, 64, 3, 3)
    bias=layer_info[2]['bias'],                   # (128,)
    x_scale=layer_info[2]['x_scale'].item(),      # scalar
    x_zp=layer_info[2]['x_zp'].item(),            # scalar
    w_scale=layer_info[2]['w_scale'],             # (128,)
    w_zp=layer_info[2]['w_zp'],                   # (128,)
    y_scale=layer_info[2]['y_scale'].item(),      # scalar
    y_zp=layer_info[2]['y_zp'].item(),            # scalar
    padding=1
)

print("✅ Layer 3 Output Shape:", layer3_out_q.shape)  # Should be (128, 112, 112)


✅ Layer 3 Output Shape: (128, 112, 112)


In [90]:
# === Layer 4 Convolution ===
layer4_out_q = manual_qconv2d(
    input_q=layer3_out_q,                        # (128, 112, 112)
    W_q=layer_info[3]['W'],                      # (128, 128, 3, 3)
    bias=layer_info[3]['bias'],                  # (128,)
    x_scale=layer_info[3]['x_scale'].item(),     # scalar
    x_zp=layer_info[3]['x_zp'].item(),           # scalar
    w_scale=layer_info[3]['w_scale'],            # (128,)
    w_zp=layer_info[3]['w_zp'],                  # (128,)
    y_scale=layer_info[3]['y_scale'].item(),     # scalar
    y_zp=layer_info[3]['y_zp'].item(),           # scalar
    padding=1
)

print("✅ Layer 4 Output Shape:", layer4_out_q.shape)  # Expect (128, 112, 112)


✅ Layer 4 Output Shape: (128, 112, 112)


In [91]:
# === MaxPool after Layer 4 ===
layer4_pooled_q = maxpool2d(layer4_out_q, kernel_size=2, stride=2)
print("✅ MaxPool Output Shape after Layer 4:", layer4_pooled_q.shape)  # Expect (128, 56, 56)


✅ MaxPool Output Shape after Layer 4: (128, 56, 56)


In [92]:
# === Layer 5 (Conv after maxpool) ===
layer5_out_q = manual_qconv2d(
    input_q=layer4_pooled_q,
    W_q=layer_info[4]['W'],
    bias=layer_info[4]['bias'],
    x_scale=layer_info[4]['x_scale'],
    x_zp=layer_info[4]['x_zp'],
    w_scale=layer_info[4]['w_scale'],
    w_zp=layer_info[4]['w_zp'],
    y_scale=layer_info[4]['y_scale'],
    y_zp=layer_info[4]['y_zp'],
    padding=1  # 👈 To maintain 56x56
)

print("✅ Layer 5 Output Shape:", layer5_out_q.shape)  # Expect (256, 56, 56)


✅ Layer 5 Output Shape: (256, 56, 56)


In [93]:
# === Layer 6 (Conv) ===
layer6_out_q = manual_qconv2d(
    input_q=layer5_out_q,
    W_q=layer_info[5]['W'],
    bias=layer_info[5]['bias'],
    x_scale=layer_info[5]['x_scale'],
    x_zp=layer_info[5]['x_zp'],
    w_scale=layer_info[5]['w_scale'],
    w_zp=layer_info[5]['w_zp'],
    y_scale=layer_info[5]['y_scale'],
    y_zp=layer_info[5]['y_zp'],
    padding=1  # To preserve spatial size (56×56)
)

print("✅ Layer 6 Output Shape:", layer6_out_q.shape)  # Expect (256, 56, 56)


✅ Layer 6 Output Shape: (256, 56, 56)


In [94]:
# === Layer 7 (Conv) ===
layer7_out_q = manual_qconv2d(
    input_q=layer6_out_q,
    W_q=layer_info[6]['W'],
    bias=layer_info[6]['bias'],
    x_scale=layer_info[6]['x_scale'],
    x_zp=layer_info[6]['x_zp'],
    w_scale=layer_info[6]['w_scale'],
    w_zp=layer_info[6]['w_zp'],
    y_scale=layer_info[6]['y_scale'],
    y_zp=layer_info[6]['y_zp'],
    padding=1  # Maintain shape (56x56)
)

print("✅ Layer 7 Output Shape:", layer7_out_q.shape)  # Expect (256, 56, 56)


✅ Layer 7 Output Shape: (256, 56, 56)


In [95]:
# === MaxPool after Layer 7 ===
layer7_pooled_q = maxpool2d(layer7_out_q, kernel_size=2, stride=2)
print("✅ MaxPool Output Shape:", layer7_pooled_q.shape)  # Expect (256, 28, 28)


✅ MaxPool Output Shape: (256, 28, 28)


In [96]:
# === Layer 8 (Conv) ===
layer8_out_q = manual_qconv2d(
    input_q=layer7_pooled_q,
    W_q=layer_info[7]['W'],
    bias=layer_info[7]['bias'],
    x_scale=layer_info[7]['x_scale'],
    x_zp=layer_info[7]['x_zp'],
    w_scale=layer_info[7]['w_scale'],
    w_zp=layer_info[7]['w_zp'],
    y_scale=layer_info[7]['y_scale'],
    y_zp=layer_info[7]['y_zp'],
    padding=1  # Maintain (28, 28)
)

print("✅ Layer 8 Output Shape:", layer8_out_q.shape)  # Expect (512, 28, 28)


✅ Layer 8 Output Shape: (512, 28, 28)


In [97]:
# === Layer 9 (Conv) ===
layer9_out_q = manual_qconv2d(
    input_q=layer8_out_q,
    W_q=layer_info[8]['W'],
    bias=layer_info[8]['bias'],
    x_scale=layer_info[8]['x_scale'],
    x_zp=layer_info[8]['x_zp'],
    w_scale=layer_info[8]['w_scale'],
    w_zp=layer_info[8]['w_zp'],
    y_scale=layer_info[8]['y_scale'],
    y_zp=layer_info[8]['y_zp'],
    padding=1  # Keep output size same (28x28)
)

print("✅ Layer 9 Output Shape:", layer9_out_q.shape)  # Expect (512, 28, 28)


✅ Layer 9 Output Shape: (512, 28, 28)


In [98]:
# === Layer 10 (Conv) ===
layer10_out_q = manual_qconv2d(
    input_q=layer9_out_q,
    W_q=layer_info[9]['W'],
    bias=layer_info[9]['bias'],
    x_scale=layer_info[9]['x_scale'],
    x_zp=layer_info[9]['x_zp'],
    w_scale=layer_info[9]['w_scale'],
    w_zp=layer_info[9]['w_zp'],
    y_scale=layer_info[9]['y_scale'],
    y_zp=layer_info[9]['y_zp'],
    padding=1  # Keep output size same (28x28)
)

print("✅ Layer 10 Output Shape:", layer10_out_q.shape)  # Expect (512, 28, 28)


✅ Layer 10 Output Shape: (512, 28, 28)


In [99]:
# === MaxPool after Layer 10 ===
layer10_pooled_q = maxpool2d(layer10_out_q, kernel_size=2, stride=2)

print("✅ MaxPool Output Shape after Layer 10:", layer10_pooled_q.shape)  # Expect (512, 14, 14)


✅ MaxPool Output Shape after Layer 10: (512, 14, 14)


In [100]:
# === Layer 11 ===
layer11_out_q = manual_qconv2d(
    input_q=layer10_pooled_q,
    W_q=layer_info[10]['W'],
    bias=layer_info[10]['bias'],
    x_scale=layer_info[10]['x_scale'],
    x_zp=layer_info[10]['x_zp'],
    w_scale=layer_info[10]['w_scale'],
    w_zp=layer_info[10]['w_zp'],
    y_scale=layer_info[10]['y_scale'],
    y_zp=layer_info[10]['y_zp'],
    padding=1
)

print("✅ Layer 11 Output Shape:", layer11_out_q.shape)  # Expected: (512, 14, 14)


✅ Layer 11 Output Shape: (512, 14, 14)


In [101]:
# === Layer 12 (Conv) ===
layer12_out_q = manual_qconv2d(
    input_q=layer11_out_q,
    W_q=layer_info[11]['W'],
    bias=layer_info[11]['bias'],
    x_scale=layer_info[11]['x_scale'],
    x_zp=layer_info[11]['x_zp'],
    w_scale=layer_info[11]['w_scale'],
    w_zp=layer_info[11]['w_zp'],
    y_scale=layer_info[11]['y_scale'],
    y_zp=layer_info[11]['y_zp'],
    padding=1  # Maintain (14, 14)
)

print("✅ Layer 12 Output Shape:", layer12_out_q.shape)  # Expect (512, 14, 14)


✅ Layer 12 Output Shape: (512, 14, 14)


In [102]:
# === Layer 13 (Conv) ===
layer13_out_q = manual_qconv2d(
    input_q=layer12_out_q,
    W_q=layer_info[12]['W'],
    bias=layer_info[12]['bias'],
    x_scale=layer_info[12]['x_scale'],
    x_zp=layer_info[12]['x_zp'],
    w_scale=layer_info[12]['w_scale'],
    w_zp=layer_info[12]['w_zp'],
    y_scale=layer_info[12]['y_scale'],
    y_zp=layer_info[12]['y_zp'],
    padding=1  # Keep output size at 14x14
)

print("✅ Layer 13 Output Shape:", layer13_out_q.shape)  # Expect (512, 14, 14)


✅ Layer 13 Output Shape: (512, 14, 14)


In [103]:
# === MaxPool after Layer 13 ===
layer13_pooled_q = maxpool2d(layer13_out_q, kernel_size=2, stride=2)

print("✅ MaxPool Output Shape after Layer 13:", layer13_pooled_q.shape)  # Expect (512, 7, 7)


✅ MaxPool Output Shape after Layer 13: (512, 7, 7)


In [104]:
# === Flatten pooled output ===
flattened_q = layer13_pooled_q.flatten()  # Shape: (512 * 7 * 7 = 25088,)

# === Dequantize before FC ===
x_scale_fc = 0.27242821
x_zp_fc = 0

flattened_f32 = x_scale_fc * (flattened_q.astype(np.float32) - x_zp_fc)

print("✅ Dequantized Flattened Output Shape:", flattened_f32.shape)  # (25088,)


✅ Dequantized Flattened Output Shape: (25088,)


In [106]:
import onnx
from onnx import numpy_helper

# Load model
model_path = "vgg16_int8.onnx"
onnx_model = onnx.load(model_path)

# Build initializer map
initializer_map = {init.name: init for init in onnx_model.graph.initializer}
def get_array(name): return numpy_helper.to_array(initializer_map[name])

# List all initializers (to locate FC layers)
print("All Initializers:")
for name in initializer_map:
    print(name, get_array(name).shape)

# === Extract FC Weights and Biases ===

# These are likely the FC layers; replace names if needed based on the above printout
fc_names = {
    "fc1": {
        "W": "fc.weight1",   # likely: (25088, 4096)
        "B": "fc.bias1"
    },
    "fc2": {
        "W": "fc.weight2",   # likely: (4096, 4096)
        "B": "fc.bias2"
    },
    "fc3": {
        "W": "fc.weight3",   # likely: (4096, 1000)
        "B": "fc.bias3"
    }
}

fc_weights = {}
for fc, names in fc_names.items():
    try:
        W = get_array(names["W"])
        B = get_array(names["B"])
        fc_weights[fc] = {"W": W, "B": B}
        print(f"{fc.upper()} -> W: {W.shape}, B: {B.shape}")
    except KeyError as e:
        print(f"⚠️ Missing: {e}")

# Now `fc_weights['fc1']['W']` gives FC1 weights, etc.


All Initializers:
data_zero_point ()
data_scale ()
vgg0_conv0_weight_quantized (64, 3, 3, 3)
vgg0_conv0_weight_scale (64,)
vgg0_conv0_weight_zero_point (64,)
vgg0_conv0_bias_quantized (64,)
vgg0_conv0_fwd_zero_point ()
vgg0_conv0_fwd_scale ()
vgg0_conv1_weight_quantized (64, 64, 3, 3)
vgg0_conv1_weight_scale (64,)
vgg0_conv1_weight_zero_point (64,)
vgg0_conv1_bias_quantized (64,)
vgg0_conv1_fwd_zero_point ()
vgg0_conv1_fwd_scale ()
vgg0_conv2_weight_quantized (128, 64, 3, 3)
vgg0_conv2_weight_scale (128,)
vgg0_conv2_weight_zero_point (128,)
vgg0_conv2_bias_quantized (128,)
vgg0_conv2_fwd_zero_point ()
vgg0_conv2_fwd_scale ()
vgg0_conv3_weight_quantized (128, 128, 3, 3)
vgg0_conv3_weight_scale (128,)
vgg0_conv3_weight_zero_point (128,)
vgg0_conv3_bias_quantized (128,)
vgg0_conv3_fwd_zero_point ()
vgg0_conv3_fwd_scale ()
vgg0_conv4_weight_quantized (256, 128, 3, 3)
vgg0_conv4_weight_scale (256,)
vgg0_conv4_weight_zero_point (256,)
vgg0_conv4_bias_quantized (256,)
vgg0_conv4_fwd_zero_poin

In [107]:
fc_layer_info = []

fc_names = ["dense0", "dense1", "dense2"]

for i, fc in enumerate(fc_names):
    layer = {}
    prefix = f"vgg0_{fc}_"

    # Weight and bias
    layer["W"] = get_array(prefix + "weight_quantized")          # int8 weights
    layer["w_scale"] = get_array(prefix + "weight_scale")        # float32
    layer["w_zp"] = get_array(prefix + "weight_zero_point")      # int8 or uint8
    
    layer["bias"] = get_array(prefix + "bias_quantized")         # int8 bias
    layer["bias_scale"] = get_array(prefix + "bias_scale")
    layer["bias_zp"] = get_array(prefix + "bias_zero_point")

    # Output (MatMul) quant params
    layer["out_scale"] = get_array(prefix + "fwd_MatMul_scale")
    layer["out_zp"] = get_array(prefix + "fwd_MatMul_zero_point")

    # Final output of layer (after add)
    layer["y_scale"] = get_array(prefix + "fwd_scale")
    layer["y_zp"] = get_array(prefix + "fwd_zero_point")

    fc_layer_info.append(layer)

# ✅ Summary
for i, layer in enumerate(fc_layer_info):
    print(f"\n🔹 FC Layer {i+1}")
    print(f"  Weight shape: {layer['W'].shape}")
    print(f"  Bias shape: {layer['bias'].shape}")
    print(f"  Weight scale: {layer['w_scale']}")
    print(f"  Bias scale: {layer['bias_scale']}")
    print(f"  Output scale (MatMul): {layer['out_scale']}")
    print(f"  Final output scale (post-add): {layer['y_scale']}")



🔹 FC Layer 1
  Weight shape: (25088, 4096)
  Bias shape: (4096,)
  Weight scale: 0.0003788646135944873
  Bias scale: 0.0008186898776330054
  Output scale (MatMul): 0.2407444268465042
  Final output scale (post-add): 0.09747663140296936

🔹 FC Layer 2
  Weight shape: (4096, 4096)
  Bias shape: (4096,)
  Weight scale: 0.0004914611927233636
  Bias scale: 0.0006801208946853876
  Output scale (MatMul): 0.15741786360740662
  Final output scale (post-add): 0.05860837176442146

🔹 FC Layer 3
  Weight shape: (4096, 1000)
  Bias shape: (1000,)
  Weight scale: 0.0010433332063257694
  Bias scale: 0.0007945160032249987
  Output scale (MatMul): 0.2589195668697357
  Final output scale (post-add): 0.25890347361564636


In [108]:
def run_fc_layer(x_float, W_q, w_scale, w_zp, bias_q, bias_scale, bias_zp,
                 input_scale, input_zp, out_scale_matmul, out_zp_matmul,
                 out_scale_final, out_zp_final):
    """
    x_float: float32 input vector
    W_q: int8 weight matrix
    bias_q: int8 bias vector
    Returns final float32 output after quantized matmul + bias + dequant
    """
    # Step 1: Quantize input
    x_q = np.clip(np.round(x_float / input_scale) + input_zp, 0, 255).astype(np.uint8)

    # Step 2: QLinearMatMul
    x_int32 = x_q.astype(np.int32) - input_zp
    W_int32 = W_q.astype(np.int32) - w_zp
    out_int32 = np.dot(x_int32, W_int32)

    # Step 3: Requantize to intermediate output (same as ONNX QLinearMatMul)
    result_q = np.round(out_int32 * (input_scale * w_scale / out_scale_matmul) + out_zp_matmul).astype(np.uint8)

    # Step 4: QLinearAdd (bias)
    A = result_q.astype(np.int32) - out_zp_matmul
    B = bias_q.astype(np.int32) - bias_zp
    C_int32 = A + B  # In ONNX, both A and B are in same scale domain

    # Step 5: Final quantization
    final_q = np.round(C_int32 * (out_scale_matmul / out_scale_final) + out_zp_final)
    final_q = np.clip(final_q, 0, 255).astype(np.uint8)

    # Step 6: Dequantize
    final_float = out_scale_final * (final_q.astype(np.float32) - out_zp_final)
    return final_float


In [109]:
# FC1 parameters
W1 = fc_layer_info[0]['W']
B1 = fc_layer_info[0]['bias']
out1 = run_fc_layer(
    x_float=flattened_f32,
    W_q=W1, w_scale=0.0003788646, w_zp=0,
    bias_q=B1, bias_scale=0.0008186898, bias_zp=128,
    input_scale=0.27242821, input_zp=0,
    out_scale_matmul=0.24074442, out_zp_matmul=152,
    out_scale_final=0.09747663, out_zp_final=0
)

# FC2
W2 = fc_layer_info[1]['W']
B2 = fc_layer_info[1]['bias']
out2 = run_fc_layer(
    x_float=out1,
    W_q=W2, w_scale=0.0004914612, w_zp=0,
    bias_q=B2, bias_scale=0.0006801208, bias_zp=128,
    input_scale=0.09747663, input_zp=0,
    out_scale_matmul=0.15741786, out_zp_matmul=0,
    out_scale_final=0.05860837, out_zp_final=0
)

# FC3
W3 = fc_layer_info[2]['W']
B3 = fc_layer_info[2]['bias']
out3 = run_fc_layer(
    x_float=out2,
    W_q=W3, w_scale=0.0010433332, w_zp=0,
    bias_q=B3, bias_scale=0.0007945160, bias_zp=128,
    input_scale=0.05860837, input_zp=0,
    out_scale_matmul=0.25891956, out_zp_matmul=0,
    out_scale_final=0.25890347, out_zp_final=0
)

print("✅ Final output logits shape:", out3.shape)  # Should be (1000,)


✅ Final output logits shape: (1000,)


In [110]:
def softmax(logits):
    exps = np.exp(logits - np.max(logits))  # subtract max for numerical stability
    return exps / np.sum(exps)

# Apply softmax
probs = softmax(out3)

# Top-1 and Top-5 predictions
top1 = np.argmax(probs)
top5 = np.argsort(probs)[-5:][::-1]

print("🔮 Top-1 Class Index:", top1)
print("🔝 Top-5 Class Indices:", top5)
print("📊 Top-5 Probabilities:", probs[top5])


🔮 Top-1 Class Index: 21
🔝 Top-5 Class Indices: [333 876 474 664 965]
📊 Top-5 Probabilities: [0.02323296 0.02323296 0.02323296 0.02323296 0.02323296]
